In [15]:
import numpy as np
import os
import subprocess
import mplbm_utils as mplbm

In [16]:
def create_flat_surface(inputs):

    domain = np.zeros((100, 100, 100), dtype=np.uint8)  # empty space
    # domain[0:5,:,:] = 1  # surface
    # domain = np.transpose(domain, [1,0,2])
    domain.tofile(f"{inputs['input output']['input folder']}empty_box.raw")
    print(domain.shape)

    print('Creating efficient geometry for Palabos...')
    mplbm.create_geom_for_palabos(inputs)
    # create_geom_for_palabos(inputs)

    # import vedo as vd
    # vp = vd.Plotter(axes=3, bg='w', bg2='w', size=(1200, 900), offscreen=False)
    # domain = vd.Volume(domain).isosurface(0.5)
    # vp += domain.c('gray')
    # vp.show()

    return

In [17]:
def run_2_phase_sim(inputs):
    # Steps
    # 1) create palabos input file
    # 2) run 2-phase sim

    if inputs['simulation type'] == '1-phase':
        raise KeyError('Simulation type set to 1-phase...please change to 2-phase.')
    sim_directory = inputs['input output']['simulation directory']

    # 1) Create simulation input file
    print('Creating input file...')
    mplbm.create_palabos_input_file(inputs)

    # 2) Run 2-phase simulation
    print('Running 2-phase simulation...')
    num_procs = inputs['simulation']['num procs']
    input_dir = inputs['input output']['input folder']
    simulation_command = f"mpirun -np {num_procs} ../../src/2-phase_LBM/ShanChen {input_dir}2_phase_sim_input.xml  > {sim_directory}/{input_dir}running_script.out"
    file = open(f'{sim_directory}/{input_dir}run_shanchen_sim.sh', 'w')
    file.write(f'{simulation_command}')
    file.close()
    
    # print(f'bash {sim_directory}/{input_dir}run_shanchen_sim.sh > "{sim_directory}/{input_dir}running_script.out"')
    # return

    simulation_command_subproc = f'bash {sim_directory}/{input_dir}run_shanchen_sim.sh'
    subprocess.run(simulation_command_subproc.split(' '))

    return

# Main part

In [37]:
input_file = 'input.yml'
inputs = mplbm.parse_input_file(input_file)  # Parse inputs
inputs['input output']['simulation directory'] = os.getcwd()  # Store current working directory



G_ww = np.array([inputs['simulation']['fluid data']['G_ww']])  # G_ads_f1_s1 values corresponding to 156 deg, 90 deg, and 24 deg contact angles
G_ow = np.array([inputs['simulation']['fluid data']['G_ow']])


input_folders = []
output_folders = []

for i in range(len(G_ww)):
    for j in range(len(G_ow)):
        input_folders.append(f'input_Gww_{G_ww[i]}_Gow_{G_ow[j]}/')
        output_folders.append(f'tmp_Gww_{G_ww[i]}_Gow_{G_ow[j]}/')
# input_folders = ['Gc_0.4/', 'Gc_0.8/', 'Gc_1.5/']
# tmp_folders = ['tmp_0.4/', 'tmp_0.8/', 'tmp_1.5/']

# tmp_folders = ['tmp_m0.4/', 'tmp_0/', 'tmp_p0.4/']

for i in range(len(G_ww)):
    for j in range(len(G_ow)):
        if not os.path.exists(input_folders[i * j + j]):
            os.makedirs(input_folders[i * j + j])

        if not os.path.exists(output_folders[i * j + j]):
            os.makedirs(output_folders[i * j + j])

        inputs['input output']['input folder'] = input_folders[i * j + j]
        inputs['input output']['output folder'] = output_folders[i * j + j]
        # inputs['simulation']['fluid data']['G_ww'] = G_ww[i]
        # inputs['simulation']['fluid data']['G_ow'] = G_ow[j]
        create_flat_surface(inputs)
        print(f"\n\nRunning droplet test with G_ww = {G_ww[i]}... G_ow = {G_ow[j]}...")
        run_2_phase_sim(inputs)  # Run 2 phase sim
        print("\n\nrun_2_phase_sim DONE!!!!")

Simulation Type: 2-phase
Simulation Directory: /mnt/data2/windowkim/hyundai/lbm/MPLBM-UT/examples/pure_bubble
Input Folder: input/
Output Folder: tmp/
Geometry File: empty_box.raw
Geometry Data Type: int8
Geometry Size: Nx=100, Ny=100, Nz=100
Geometry Name: empty_box
Domain Size: nx=100, ny=100, nz=100
Restart Simulation: False
Fluid 1 Equilibrium Density: 2
Fluid 2 Equilibrium Density: 2
Fluid 1 Force: 0
Fluid 2 Force: 0
Using Pressure Boundary Conditions: False
Initial fluid configuration setup with custom settings.
Save simulation state: False
Saving density vtks every 100 iterations.
Saving density fluid 1 and fluid 2 vtks: True
(100, 100, 100)
Creating efficient geometry for Palabos...


Running droplet test with G_ww = -0.3... G_ow = 0.1...
Creating input file...
Running 2-phase simulation...


Authorization required, but no authorization protocol specified
Authorization required, but no authorization protocol specified


KeyboardInterrupt: 

In [5]:
os.getcwd()

'/mnt/data2/windowkim/hyundai/lbm/MPLBM-UT/examples/pure_bubble'